In [18]:
import numpy as np
import random
import math
import csv
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

In [19]:
music_class_binary = {'BIG_BAND':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
          'BLUES_CONTEMPORARY':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
          'COUNTRY_TRADITIONAL': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
          'DANCE':               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
          'ELECTRONICA':         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
          'EXPERIMENTAL':        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
          'FOLK_INTERNATIONAL':  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          'GOSPEL':              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
          'GRUNGE_EMO':          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          'HIP_HOP_RAP':         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'JAZZ_CLASSIC':        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'METAL_ALTERNATIVE':   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'METAL_DEATH':         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'METAL_HEAVY':         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'POP_CONTEMPORARY':    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'POP_INDIE':           [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'POP_LATIN':           [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'PUNK':                [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'REGGAE':              [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'RNB_SOUL':            [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'ROCK_ALTERNATIVE':    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'ROCK_COLLEGE':        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'ROCK_CONTEMPORARY':   [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'ROCK_HARD':           [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          'ROCK_NEO_PSYCHEDELIA':[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
music_class = {'BIG_BAND':1,
          'BLUES_CONTEMPORARY':2,
          'COUNTRY_TRADITIONAL':3,
          'DANCE':4,
          'ELECTRONICA':5,
          'EXPERIMENTAL':6,
          'FOLK_INTERNATIONAL':7,
          'GOSPEL':8,
          'GRUNGE_EMO':9,
          'HIP_HOP_RAP':10,
          'JAZZ_CLASSIC':11,
          'METAL_ALTERNATIVE':12,
          'METAL_DEATH':13,
          'METAL_HEAVY':14,
          'POP_CONTEMPORARY':15,
          'POP_INDIE':16,
          'POP_LATIN':17,
          'PUNK':18,
          'REGGAE':19,
          'RNB_SOUL':20,
          'ROCK_ALTERNATIVE':21,
          'ROCK_COLLEGE':22,
          'ROCK_CONTEMPORARY':23,
          'ROCK_HARD':24,
          'ROCK_NEO_PSYCHEDELIA':25}

In [20]:
def read_csv(url):
    """
    Args:
        url (string): the url of the file
    Returns:
        df: the dataframe filled
    """
    df = pd.read_csv(url, header=None)
    df.head()
    return df

In [21]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

In [31]:
def SVM(X_train,Y_train,X_test,Y_test,weight) :
    """
    Args:
        X_train (list): the train sample
        X_test (list): the test sample
        Y_train (list): the train output
        Y_test (list): the test output
    Returns:
        
    """
    acc_scorer = make_scorer(average_precision_score)
    scores = {'Accuracy': acc_scorer}
    
    #model  = svm.SVC(gamma='scale', decision_function_shape='ovo', kernel = 'rbf',C=10)
    #model.fit(X_train, Y_train)
    
    svc=SVC(probability=True, gamma='scale', decision_function_shape='ovo', kernel = 'rbf',C=10)
    abc =AdaBoostClassifier(n_estimators=20, base_estimator=svc,learning_rate=1)
    model = abc.fit(X_train, Y_train)
    
    #print("Debut du grid search")
    # implemente le gridsearchcv
    #parameters = {'class_weight':weight,'cache_size':[6000],'C':[0.001,0.1,1,10],'gamma':[0.001,0.1,1,10],'kernel':('linear', 'rbf')}
    #clf = GridSearchCV(model, parameters, scoring='roc_auc',n_jobs=2,cv=3)
    #clf = GridSearchCV(model, parameters,n_jobs=3,cv=3)
    #clf.fit(X_train, Y_train)
    # sort le best model pour fit et score dessus
    #best_model = clf.best_estimator_
    #print(best_model)
      
    #print("Score of the SVM model:   " + str(model.score(X_test,Y_test)))
    
    return model

In [32]:
def encode_labels(Y) :
    """
    Args:
        X_train (list): the train sample
        X_test (list): the test sample
        Y_train (list): the train output
        Y_test (list): the test output
    Returns:
        
    """
    encoder = preprocessing.LabelEncoder()
    encoder.fit(Y)
    Y_train_l = encoder.transform(Y)
    return Y_train_l

In [28]:
df = read_csv(r"C:\Users\gaspa\OneDrive\Bureau\COURS\ING 5\Semestre 1\Machine Learning\GTI770\labs\lab4\music\tagged_feature_sets\msd-ssd_dev\msd-ssd_dev.csv")

name_df = df[[0,1]]
labels = df.loc[:,df.columns == 170]
labels = labels.to_numpy()
y = []
df = df.drop(170,axis=1)

for e in labels:
    y.append(music_class[e[0]])
y_df=pd.DataFrame(y)
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:,2:169], y_df, test_size=0.9,stratify=y_df)

In [33]:
# scale the data : réduire le execution time
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

# Create an LDA that will reduce the data down to 1 feature
lda = LinearDiscriminantAnalysis(n_components=None)
# run an LDA and use it to transform the features
lda.fit(X_train, Y_train)
nb_components = select_n_components(lda.explained_variance_ratio_, 0.95)
print(nb_components)

# dimensional reduction
lda = LinearDiscriminantAnalysis(n_components=nb_components)
lda.fit(X_train, Y_train)
print(X_train.shape)
X_train = lda.transform(X_train)
X_test = lda.transform(X_test)
print(X_train.shape)

# encoding train labels and test labels
Y_train = encode_labels(Y_train)
Y_test = encode_labels(Y_test)

10
(17955, 14)
(17955, 10)


In [ ]:
# make the SVM model
model = SVM(X_train,Y_train,X_test,Y_test,['balanced'])
predictions = model.predict(X_test)

In [16]:
"""
df = read_csv(r"C:\Users\gaspa\OneDrive\Bureau\COURS\ING 5\Semestre 1\Machine Learning\GTI770\labs\lab4\music\tagged_feature_sets\msd-jmirmfccs_dev\msd-jmirmfccs_dev.csv")

name_df = df[[0,1]]
labels = df.loc[:,df.columns == 28]
labels = labels.to_numpy()
y = []
df = df.drop(28,axis=1)

for e in labels:
    y.append(music_class[e[0]])
y_df=pd.DataFrame(y)
X_train, X_test, Y_train, Y_test = train_test_split(df.iloc[:,2:27], y_df, test_size=0.3,stratify=y_df)
"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 19-20: truncated \UXXXXXXXX escape (<ipython-input-16-461024361bda>, line 14)